Trying LCEL

In [1]:
pip install langchain bs4 sentence_transformers --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


In [3]:
model = Ollama(model='mistral')


embedder_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embedder = HuggingFaceBgeEmbeddings(
    model_name=embedder_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

url = "https://en.wikipedia.org/wiki/Cyclone_Michaung"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_loader = WebBaseLoader(url)

split_data = text_splitter.split_documents(text_loader.load())

print(split_data)

vectorstore = Chroma.from_documents(documents=split_data, embedding=embedder)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content='Cyclone Michaung - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\nLanguages\n\nLanguage links are at the top of the page across from the title.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in', metadata={'source': 'https://en.wikipedia.org/wiki/Cyclone_Michaung', 'title': 'Cyclone Michaung - Wikipedia', 'language': 'en'}), Document(page_content='Create account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n

In [20]:
template = """
You are a news expert who answers questions about news.  

It is important that you use the most recent sources possible using this context:
{context}

You must name any sources at the end of your answer in the following format: 
"Sources: \n[source name, article title, date published]"
Do NOT povide any urls.

If you don't have the relevent information to answer the question, tell the user
so.

If the question is not about news, tell the user "Sorry, I can only answer 
questions about news".

Here is the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = setup_and_retrieval | prompt | model | output_parser

In [24]:
print(chain.invoke("How do I drive a car?"))

I apologize, but I cannot answer the question "How do I drive a car?" as it is not related to news. If you have any other questions, please let me know and I will do my best to assist you.
